0. 所有的题目结果中，给出SQL语句和执行结果。

1. 在新数据库中新建一张 user 表,插入几条数据,属性包含:唯一标识(id),姓名(name)性别(sex).年龄(age).联系方式(phone)，数据如下：

    ('John Doe', 'Male', 25, '123-456-7890')

    ('Jane Smith', 'Female', 31, '987-654-3210')
    
    ('Bob Johnson', 'Male', 22, '555-123-4567')

In [ ]:
CREATE TABLE user (
    id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(255),
    sex VARCHAR(10),
    age INT,
    phone VARCHAR(20)
);

INSERT INTO user (name, sex, age, phone) VALUES
('John Doe', 'Male', 25, '123-456-7890'),
('Jane Smith', 'Female', 31, '987-654-3210'),
('Bob Johnson', 'Male', 22, '555-123-4567');


2. 写出 SQL语句,查询 user 表中所有年龄在 20-30 范围内的用户

In [ ]:
SELECT * FROM user WHERE age BETWEEN 20 AND 30;

3. 写出SQL语句，向user表中添加自己的个人信息，并添加几条和你姓名同姓的虚拟信息。

In [ ]:
INSERT INTO user (name, sex, age, phone) VALUES
('Alice Wang', 'Female', 28, '111-222-3333'),
('Tom Lee', 'Male', 24, '444-555-6666'),
('Lucy Zhang', 'Female', 29, '777-888-9999');


4. 写出 SQL语句,查询 user 表中年龄在 20-30 范围内,名字包含“你的姓氏”的用户,并按照年龄从大到小排序输出

In [ ]:
SELECT * FROM user WHERE age BETWEEN 20 AND 30 AND name LIKE '%Wang%' ORDER BY age DESC;

5. 写出 SQL 语句,计算 user 表中所有用户的平均年龄

In [ ]:
SELECT AVG(age) AS average_age FROM user;

6. 新建两张表team 表(id,teamName)和score 表(id,teamid,userid,score)。其中score 表中的 teamid 为指向 team表id 的外键，userid 为指向 user表id的外键

In [ ]:
CREATE TABLE team (
    id INT AUTO_INCREMENT PRIMARY KEY,
    teamName VARCHAR(255)
);

CREATE TABLE score (
    id INT AUTO_INCREMENT PRIMARY KEY,
    teamid INT,
    userid INT,
    score INT,
    FOREIGN KEY (teamid) REFERENCES team(id),
    FOREIGN KEY (userid) REFERENCES user(id)
);

INSERT INTO team (teamName) VALUES ('ECNU');
INSERT INTO score (teamid, userid, score) VALUES
(1, 1, NULL), -- Assuming ids from user table are used here
(1, 2, 85),
(1, 3, 90);

7. 在team表中和score表中插入合适的记录，写出 SQL语句,查询 teamName 为“ECNU”的队伍中，年龄小于 20 的用户们，结果不得为空。

In [ ]:
SELECT u.* FROM user u
JOIN score s ON u.id = s.userid
JOIN team t ON s.teamid = t.id
WHERE t.teamName = 'ECNU' AND u.age < 20;


8. 写出 SQL 语句,计算 teamName为“ECNU”的总分(假设 score 存在 null值,nul值默认为 0 加入计算)。

In [ ]:
SELECT SUM(COALESCE(score, 0)) AS total_score FROM score
JOIN team ON score.teamid = team.id
WHERE team.teamName = 'ECNU';


9. 写出SQL语句，删除user表中个人信息的记录。

In [ ]:
DELETE FROM user;

In [1]:
import pymysql

# 连接到数据库
connection = pymysql.connect(host='localhost',
                             user='yourusername',
                             password='yourpassword',
                             database='yourdatabase')

try:
    with connection.cursor() as cursor:
        # 创建 user 表并插入数据
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS user (
                id INT AUTO_INCREMENT PRIMARY KEY,
                name VARCHAR(255),
                sex VARCHAR(10),
                age INT,
                phone VARCHAR(20)
            );
        """)
        cursor.executemany("""
            INSERT INTO user (name, sex, age, phone) VALUES (%s, %s, %s, %s);
        """, [
            ('John Doe', 'Male', 25, '123-456-7890'),
            ('Jane Smith', 'Female', 31, '987-654-3210'),
            ('Bob Johnson', 'Male', 22, '555-123-4567')
        ])
        connection.commit()
        
        # 查询 user 表中所有年龄在 20-30 范围内的用户
        cursor.execute("SELECT * FROM user WHERE age BETWEEN 20 AND 30;")
        result = cursor.fetchall()
        print("Users aged between 20 and 30:", result)
        
        # 向 user 表中添加个人信息和虚拟信息
        cursor.executemany("""
            INSERT INTO user (name, sex, age, phone) VALUES (%s, %s, %s, %s);
        """, [
            ('Alice Wang', 'Female', 28, '111-222-3333'),
            ('Tom Lee', 'Male', 24, '444-555-6666'),
            ('Lucy Zhang', 'Female', 29, '777-888-9999')
        ])
        connection.commit()
        
        # 查询 user 表中年龄在 20-30 范围内，名字包含“你的姓氏”的用户，并按照年龄从大到小排序输出
        cursor.execute("SELECT * FROM user WHERE age BETWEEN 20 AND 30 AND name LIKE '%Wang%' ORDER BY age DESC;")
        result = cursor.fetchall()
        print("Users aged between 20 and 30 with surname 'Wang':", result)
        
        # 计算 user 表中所有用户的平均年龄
        cursor.execute("SELECT AVG(age) AS average_age FROM user;")
        result = cursor.fetchone()
        print("Average age of users:", result['average_age'])
        
        # 创建 team 表和 score 表，并插入记录
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS team (
                id INT AUTO_INCREMENT PRIMARY KEY,
                teamName VARCHAR(255)
            );
        """)
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS score (
                id INT AUTO_INCREMENT PRIMARY KEY,
                teamid INT,
                userid INT,
                score INT,
                FOREIGN KEY (teamid) REFERENCES team(id),
                FOREIGN KEY (userid) REFERENCES user(id)
            );
        """)
        cursor.execute("INSERT INTO team (teamName) VALUES ('ECNU');")
        cursor.executemany("""
            INSERT INTO score (teamid, userid, score) VALUES (%s, %s, %s);
        """, [
            (1, 1, None), # Assuming ids from user table are used here
            (1, 2, 85),
            (1, 3, 90)
        ])
        connection.commit()
        
        # 查询 teamName 为 “ECNU”的队伍中，年龄小于 20 的用户们，结果不得为空
        cursor.execute("""
            SELECT u.* FROM user u
            JOIN score s ON u.id = s.userid
            JOIN team t ON s.teamid = t.id
            WHERE t.teamName = 'ECNU' AND u.age < 20;
        """)
        result = cursor.fetchall()
        print("Users in team 'ECNU' who are younger than 20:", result)
        
        # 计算 teamName 为 “ECNU”的总分（假设 score 存在 null值，null值默认加入0）
        cursor.execute("""
            SELECT SUM(COALESCE(score, 0)) AS total_score FROM score
            JOIN team ON score.teamid = team.id
            WHERE team.teamName = 'ECNU';
        """)
        result = cursor.fetchone()
        print("Total score for team 'ECNU':", result['total_score'])
        
        # 删除 user 表中的所有个人信息记录
        cursor.execute("DELETE FROM user;")
        connection.commit()
finally:
    connection.close()


RuntimeError: 'cryptography' package is required for sha256_password or caching_sha2_password auth methods